### Introduction

This tutorial is used to show how we could use **GCP service** to do some common tasks with **Machine Learning** trained models by **Google**. So the prediction is highly confident by some use case than the models trained by ourself, is there any company has more data than google?

So this tutorial is try to use common API provided by cloud, like **translate**, **computer vision**, **speech to text** etc.

I have to say that if you need to use this API, so we do need to **enable** the API that we use.

Let's start.

In [3]:
# auth this notebook
from google.colab import auth
auth.authenticate_user()

In [4]:
# set project
! gcloud config set project cloudtutorial-279003

Updated property [core/project].


### Translate in cloud


In [4]:
# first let's install API, currently we use 2.0.0, so support other features.
# ! pip install google-cloud-translate==2.0.0 --quiet

In [13]:
# before we use the API, we have to provide with the credentials
# I just upload the credential file into server
# then just set the application
import json
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = [x for x in os.listdir('.') if 'cloud' in x.lower()][0]

In [7]:
# let's try to use this API
from google.cloud import translate_v2 as translate
import six

project_id = "cloudtutorial-279003"

# init client
client = translate.Client(project_id)

In [8]:
text = "machine learning is great"

if isinstance(text, six.binary_type):
  text = text.decode('utf-8')

# try to use client
result = client.translate(text, target_language='zh')

# so you could see that we do translate from english into chinese.
for key, value in result.items():
  print("Key: {}, value: {}".format(key, value))

Key: translatedText, value: 机器学习很棒
Key: detectedSourceLanguage, value: en
Key: input, value: machine learning is great


In [14]:
# When I use High level translate, I always face error:AttributeError: 'str' object has no attribute 'translate_text'
# so according: https://stackoverflow.com/questions/52777183/attributeerror-str-object-has-no-attribute-before-request
# change credential type

from google.oauth2 import service_account

credential_file = [x for x in os.listdir('.') if 'cloud' in x.lower()][0]
print("Credential file: {}".format(credential_file))

credentials = service_account.Credentials.from_service_account_file(credential_file)

Credential file: cloudtutorial-279003-4baef17209a9.json


In [10]:
# when we want to use higher level API, we need to install translate API
# and restart the kernel
! pip install google-cloud-translate==2.0.0 --quiet

     |████████████████████████████████| 92kB 4.7MB/s 


In [11]:
# let's wrap the logic into function, so that we could use it more easy
from google.cloud import translate
import six

def trans_text(text, project_id="cloudtutorial-279003", target_language='zh'):
  # this is to add credential to init the client.
  client = translate.TranslationServiceClient(credentials=credentials)

  parent = client.location_path(project_id, "global")

  # we could also send 'text/html' into the server.
  response = client.translate_text(
        parent=parent,
        contents=[text],
        mime_type="text/plain",  # mime types: text/plain, text/html
        source_language_code="en-US",
        target_language_code="zh",
    )
  
  res_list = []
  for translation in response.translations:
    res_list.append(translation.translated_text)

  return res_list

In [12]:
# let's check this function
res = trans_text( "today I have drunk some coffee")

print("Get result: ", res)

Get result:  ['今天我喝了咖啡']


### Vision

As we have get some info about the translate, now let's try with image vision to see what we could do with **Vision**.

In [32]:
# first let's install the library
! pip install --upgrade google-cloud-vision --quiet

In [48]:
# let's first try with OCR
from google.cloud import vision
import io


def detect_text(file_path):
  # init client
  client = vision.ImageAnnotatorClient()

  # read image
  with io.open(file_path, 'rb') as f:
    content = f.read()

  # then let's put the image content into vision type image
  image = vision.types.Image(content=content)

  # then we could send our image into remote server
  response = client.text_detection(image)
  texts = response.text_annotations

  # out = list()
  # for text in texts:
  #   out_dict = dict()
  #   out_dict['{}'.format(text.description)] = text.description
  #   bounds = ",".join(['({},{})'.format(v.x, v.y) for v in text.bounding_poly.vertices])
  #   out_dict['bounds'] = bounds

  # just to print out
  print("GET text:", end='\n')

  for text in texts:
    print("{}".format(text.description))
    bound = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])
    print("bounds: {}".format(','.join(bound)))

  if response.error.message:
    print("When to use vision get error:", response.error.message)

  print("Vision finished")

In [51]:
image_path = './vision_image.jpg'

out = detect_text(image_path)

GET text:
AJulie's.
WAFERS
Chocolate Hazelnut
CREAM-FILLED WAFER CUBES
E.
Pa化餅動
FREE
东蒂丝

bounds: (131,390),(996,390),(996,1215),(131,1215)
AJulie's.
bounds: (335,390),(509,450),(496,488),(322,428)
WAFERS
bounds: (187,434),(551,541),(519,648),(155,541)
Chocolate
bounds: (163,576),(329,624),(315,673),(149,625)
Hazelnut
bounds: (340,630),(515,680),(502,724),(327,674)
CREAM-FILLED
bounds: (139,641),(310,687),(302,717),(131,671)
WAFER
bounds: (326,695),(398,715),(392,737),(320,717)
CUBES
bounds: (414,721),(490,742),(485,763),(408,742)
E.
bounds: (752,960),(996,1009),(964,1169),(720,1120)
Pa
bounds: (150,856),(165,870),(153,883),(138,869)
化
bounds: (164,878),(174,887),(165,896),(156,887)
餅
bounds: (178,888),(188,897),(177,908),(168,899)
動
bounds: (195,899),(205,908),(195,918),(186,909)
FREE
bounds: (732,1104),(965,1148),(952,1215),(719,1171)
东
bounds: (531,431),(549,435),(544,459),(526,455)
蒂丝
bounds: (525,457),(544,461),(536,497),(517,493)
Vision finished


### Image to text -> text-translate -> text-to-speech

Bellow I will try to combine **image to text**, **translate** and **text to speech**, so that we could get image info by speech.

Let's get start.

In [13]:
# first install
! pip install --upgrade google-cloud-vision --quiet
! pip install --upgrade google-cloud-texttospeech --quiet
! pip install --upgrade google-cloud-translate --quiet

     |████████████████████████████████| 440kB 7.5MB/s 
     |████████████████████████████████| 61kB 3.8MB/s 
     |████████████████████████████████| 491kB 12.5MB/s 
     |████████████████████████████████| 92kB 7.6MB/s 
     |████████████████████████████████| 276kB 23.8MB/s 
     |████████████████████████████████| 92kB 7.7MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.17.2, but you'll have google-auth 1.18.0 which is incompatible.
ERROR: google-api-core 1.21.0 has requirement protobuf>=3.12.0, but you'll have protobuf 3.10.0 which is incompatible.
ERROR: chainer 6.5.0 has requirement typing-extensions<=3.6.6, but you'll have typing-extensions 3.7.4.2 which is incompatible.
     |████████████████████████████████| 92kB 3.9MB/s 
     |████████████████████████████████| 1.3MB 14.6MB/s 
ERROR: chainer 6.5.0 has requirement typing-extensions<=3.6.6, but you'll have typing-extensions 3.7.4.2 which is incompatible.


In [14]:
# I have to reinstall texttospeech as error: AttributeError: module 'google.cloud.texttospeech' has no attribute 'types'
! pip uninstall google-cloud

# reinstall 
! pip install --upgrade google-cloud-texttospeech --quiet

In [15]:
# as face error; AttributeError: module 'google.protobuf.descriptor' has no attribute '_internal_create_key'
# so we have to uninstall protobuf first and re-install it
! pip uninstall protobuf

! pip install --upgrade protobuf

Uninstalling protobuf-3.12.2:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/google/protobuf/*
    /usr/local/lib/python3.6/dist-packages/protobuf-3.12.2-py3.6-nspkg.pth
    /usr/local/lib/python3.6/dist-packages/protobuf-3.12.2.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled protobuf-3.12.2
  Using cached https://files.pythonhosted.org/packages/28/05/9867ef8eafd12265267bee138fa2c46ebf34a276ea4cbe184cba4c606e8b/protobuf-3.12.2-cp36-cp36m-manylinux1_x86_64.whl
ERROR: chainer 6.5.0 has requirement typing-extensions<=3.6.6, but you'll have typing-extensions 3.7.4.2 which is incompatible.


In [15]:
# import 
import io
import os
import html

# google api
from google.api_core.exceptions import AlreadyExists
from google.cloud import texttospeech
from google.cloud import translate
from google.cloud import vision

In [16]:
# detect the image text
def get_text_from_pic(input_file):
  client = vision.ImageAnnotatorClient()

  # read image
  with open(input_file, 'rb') as f:
    content = f.read()
 
  # get image object
  image = vision.types.Image(content=content)

  # For dense text, use document_text_detection
  # For less dense text, use text_detection
  response = client.document_text_detection(image=image)

  text = response.full_text_annotation.text

  return text

In [17]:
# translate chinese into english
def translate_text(text, source_language='zh', target_language='en-US', 
                   project_id="cloudtutorial-279003"):
  client = translate.TranslationServiceClient()

  location = 'us-central1'

  parent = client.location_path(project_id, location)

  response = client.translate_text(parent=parent, 
                                 contents=[text], 
                                 mime_type='text/plain', 
                                 source_language_code=source_language, 
                                 target_language_code=target_language )
  
  res_list = []
  for res in response.translations:
    res_list.append(res.translated_text)

  return res_list[0]

In [18]:
# convert translated result into speech
def text_to_speech(text, output_file):
  # Replace special characters with HTML
  escaped_lines = html.escape(text)

  # Convert plaintext to SSML in order to wait 0.5 seconds
  # between each line in synthetic speech
  # so that this make it more readable
  ssml = '<speak>{}</speak>'.format(
        escaped_lines.replace('\n', '\n<break time="0.5s"/>'))

  client = texttospeech.TextToSpeechClient()

  # set text input into synthesized
  synthesis_text = texttospeech.SynthesisInput(ssml=ssml)

  # build voice request, we could set the ouput voice gender to 'NEUTRAL'
  voice = texttospeech.VoiceSelectionParams(language_code='en-US', 
                                                  ssml_gender=texttospeech.SsmlVoiceGender.NEUTRAL)
  
  # Selects the type of audio file to return
  audio_config = texttospeech.AudioConfig(audio_encoding=texttospeech.AudioEncoding.MP3)
  
  # put request
  response = client.synthesize_speech(input=synthesis_text, voice=voice, audio_config=audio_config)

  # Write the respone into a mp3 file
  with open(output_file, 'wb') as f:
    f.write(response.audio_content)
    print("file has been writen into file: {}".format(output_file))



In [19]:
input_file = "image_to_text.jpg"
output_file = "sample.mp3"

# photo to text
text = get_text_from_pic(input_file)

In [20]:
# let's check the text
print("Get text:{}".format(text))

Get text:但如果节点本身崩溃,那么 Control Plane 必须为所有随节点停止运行的 pod 创
建替代品。它不会为你直接创建的pod 执行此操作。这些 pod 只被 Kubelet 管理,
但由于Kubelet 本身运行在节点上,所以如果节点异常终止,它将无法执行任何操作。
为了确保你的应用程序在另一个节点上重新启动,需要使用
ReplicationController 或类似机制管理 pod,我们将在本章其余部分讨论该
机制。
4.2 了解ReplicationController
ReplicationController 是一种Kubemetes 资源,可确保它的pod 始终保持运行状态。
如果 pod 因任何原因消失(例如节点从集群中消失或由于该 pod 己从节点中逐出),
则 ReplicationController 会注意到缺少了pod 并创建替代 pod.
图4.1 显示了当一个节点下线且带有两个 pod 时会发生什么。 pod A 是被直接创
建的,因此是非托管的pod, 而 pod B由 ReplicationController 管理。节点异常退出后,



In [21]:
# let's translate into english
translated_text = translate_text(text)

print("Get translated text: {}".format(translated_text))

Get translated text: But if the node itself crashes, then the Control Plane must create for all pods that stop running with the node
Build alternatives. It will not do this for the pod you created directly. These pods are only managed by Kubelet,
But since Kubelet itself runs on the node, if the node terminates abnormally, it will not be able to perform any operations.
To ensure that your application restarts on another node, you need to use
ReplicationController or similar mechanism to manage pods, which we will discuss in the rest of this chapter
mechanism.
4.2 Understanding ReplicationController
ReplicationController is a Kubemetes resource that can ensure that its pods are always running.
If the pod disappears for any reason (for example, the node disappears from the cluster or the pod has been evicted from the node),
Then ReplicationController will notice that the pod is missing and create a replacement pod.
Figure 4.1 shows what happens when a node goes offline and has two pods. 

In [30]:
# this is just want to convert the text to be more normally.
translated_text = translated_text.replace('\n', ' ').replace('. ', '.\n')
print(translated_text)

But if the node itself crashes, then the Control Plane must create for all pods that stop running with the node Build alternatives.
It will not do this for the pod you created directly.
These pods are only managed by Kubelet, But since Kubelet itself runs on the node, if the node terminates abnormally, it will not be able to perform any operations.
To ensure that your application restarts on another node, you need to use ReplicationController or similar mechanism to manage pods, which we will discuss in the rest of this chapter mechanism.
4.2 Understanding ReplicationController ReplicationController is a Kubemetes resource that can ensure that its pods are always running.
If the pod disappears for any reason (for example, the node disappears from the cluster or the pod has been evicted from the node), Then ReplicationController will notice that the pod is missing and create a replacement pod.
Figure 4.1 shows what happens when a node goes offline and has two pods.
pod A is directly cre

In [31]:
# alright let's try to convert this text into speech with converted text
text_to_speech(translated_text, output_file)

file has been writen into file: sample.mp3
